### Import Library

In [27]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import graphviz
from sklearn.model_selection import train_test_split ## 訓練、測試切割
import matplotlib.pyplot as plt
from scipy import stats

### Data Preprocessing

In [28]:
train_data = pd.read_csv("adult/adult.data", header= None, names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])

#刪除重複的值
print("原始比數：",train_data.shape)
train_data.drop_duplicates(inplace=True)
print("刪除重複的值後：",train_data.shape)

#刪除問號的值(取眾數)
vals, counts = np.unique(train_data['workclass'],return_counts=True)
train_data['workclass'] = train_data['workclass'].replace(" ?", vals[np.argmax(counts)])
vals, counts = np.unique(train_data['occupation'],return_counts=True)
train_data['occupation'] = train_data['occupation'].replace(" ?", vals[np.argmax(counts)])
vals, counts = np.unique(train_data['native-country'],return_counts=True)
train_data['native-country'] = train_data['native-country'].replace(" ?", vals[np.argmax(counts)])

# 判斷是否還有空值
train_data.isnull().sum(axis=0)

# 刪除欄位意義重複的欄位
train_data.drop("education",axis=1,inplace=True)
train_data.drop("fnlwgt",axis=1,inplace=True)

# 每個元素的前後空格去掉
train_data = train_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

原始比數： (32561, 15)
刪除重複的值後： (32537, 15)


In [29]:
## 類別屬性做One-hot Encoding
train_nd = pd.get_dummies(train_data, columns=['workclass','marital-status','occupation','relationship','race','sex','native-country'], dtype=int)
train_nd

## 類別屬性(income)做One-hot Encoding
label_encoder = preprocessing.LabelEncoder()
train_nd["income"] = label_encoder.fit_transform(train_nd["income"])

## 數值屬性做Normalization(z-score)
numerical_columns = ['age','education-num','capital-gain','capital-loss','hours-per-week']
scaler = preprocessing.StandardScaler()
train_nd[numerical_columns] = scaler.fit_transform(train_nd[numerical_columns])

## 將"&"，改為and，以便作圖
train_nd = train_nd.rename(columns={'native-country_Trinadad&Tobago': 'native-country_Trinadad_and_Tobago',
                                    'native-country_Outlying-US(Guam-USVI-etc)': 'native-country_Outlying-US-Guam-USVI-etc'})



In [30]:
train_nd.describe()

,age,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad_and_Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
count,3.253700e+04,3.253700e+04,3.253700e+04,3.253700e+04,3.253700e+04,32537.000000,32537.000000,32537.000000,32537.000000,32537.000000,...,32537.000000,32537.000000,32537.000000,32537.000000,32537.000000,32537.000000,32537.000000,32537.000000,32537.000000,32537.000000
mean,-7.555945e-17,2.849858e-17,3.646945e-17,4.367598e-19,-6.485884e-17,0.240926,0.029505,0.064327,0.000215,0.753266,...,0.001137,0.003504,0.000369,0.002459,0.001567,0.000553,0.000584,0.913883,0.002059,0.000492
std,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,0.427652,0.169219,0.245338,0.014666,0.431117,...,0.033703,0.059089,0.019201,0.049525,0.039561,0.023514,0.024158,0.280542,0.045332,0.022170
min,-1.582777e+00,-3.531590e+00,-1.459754e-01,-2.167432e-01,-3.194403e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.761933e-01,-4.206787e-01,-1.459754e-01,-2.167432e-01,-3.566374e-02,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,-1.162616e-01,-3.181473e-02,-1.459754e-01,-2.167432e-01,-3.566374e-02,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,6.903217e-01,7.459132e-01,-1.459754e-01,-2.167432e-01,3.693028e-01,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,3.770003e+00,2.301369e+00,1.338964e+01,1.058963e+01,4.742941e+00,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
train_nd.head(8)

,age,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad_and_Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,0.030390,1.134777,0.148292,-0.216743,-0.035664,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.836973,1.134777,-0.145975,-0.216743,-2.222483,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,-0.042936,-0.420679,-0.145975,-0.216743,-0.035664,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,1.056950,-1.198407,-0.145975,-0.216743,-0.035664,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,-0.776193,1.134777,-0.145975,-0.216743,-0.035664,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,-0.116262,1.523641,-0.145975,-0.216743,-0.035664,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
6,0.763647,-1.976134,-0.145975,-0.216743,-1.979503,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,0.983625,-0.420679,-0.145975,-0.216743,0.369303,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [32]:
test_data = pd.read_csv("adult/adult.test", header= None, names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])
test_data.drop(0,axis=0,inplace=True)

#刪除重複的值
print("原始比數：",test_data.shape)
test_data.drop_duplicates(inplace=True)
print("刪除重複的值後：",test_data.shape)

#刪除問號的值(取眾數)
vals, counts = np.unique(test_data['workclass'],return_counts=True)
test_data['workclass'] = test_data['workclass'].replace(" ?", vals[np.argmax(counts)])

vals, counts = np.unique(test_data['occupation'],return_counts=True)
test_data['occupation'] = test_data['occupation'].replace(" ?", vals[np.argmax(counts)])

vals, counts = np.unique(test_data['native-country'],return_counts=True)
test_data['native-country'] = test_data['native-country'].replace(" ?", vals[np.argmax(counts)])

# 刪除欄位意義重複的欄位
test_data.drop("education",axis=1,inplace=True)
test_data.drop("fnlwgt",axis=1,inplace=True)

# 每個元素的前後空格去掉
test_data = test_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# 判斷是否還有空值
test_data.isnull().sum(axis=0)

原始比數： (16281, 15)
刪除重複的值後： (16276, 15)


age               0
workclass         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [33]:
## 類別屬性做One-hot Encoding ; test_nd(test_new_dataframe)
test_nd = pd.get_dummies(test_data,columns=['workclass','marital-status','occupation','relationship','race','sex','native-country'], dtype=int)
test_nd['native-country_Holand-Netherlands'] = 0

## 類別屬性(income)做One-hot Encoding
label_encoder = preprocessing.LabelEncoder()
test_nd["income"] = label_encoder.fit_transform(test_nd["income"])

## 數值屬性做Normalization(z-score)
numerical_columns = ['age','education-num','capital-gain','capital-loss','hours-per-week']
scaler = preprocessing.StandardScaler()
test_nd[numerical_columns] = scaler.fit_transform(test_nd[numerical_columns])

## 將"&"，改為and，以便作圖
test_nd = test_nd.rename(columns={'native-country_Trinadad&Tobago': 'native-country_Trinadad_and_Tobago',
                                  'native-country_Outlying-US(Guam-USVI-etc)': 'native-country_Outlying-US-Guam-USVI-etc'})



In [34]:
test_nd.describe()

,age,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad_and_Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,native-country_Holand-Netherlands
count,1.627600e+04,1.627600e+04,1.627600e+04,1.627600e+04,1.627600e+04,16276.000000,16276.000000,16276.000000,16276.000000,16276.000000,...,16276.000000,16276.000000,16276.000000,16276.000000,16276.000000,16276.000000,16276.000000,16276.000000,16276.000000,16276.0
mean,1.623998e-16,3.518662e-16,2.357416e-17,-4.802144e-17,-2.470922e-16,0.236299,0.029000,0.064082,0.000184,0.747665,...,0.004301,0.000553,0.002150,0.000860,0.000737,0.000492,0.917363,0.001167,0.000430,0.0
std,1.000031e+00,1.000031e+00,1.000031e+00,1.000031e+00,1.000031e+00,0.424821,0.167811,0.244907,0.013576,0.434366,...,0.065441,0.023509,0.046324,0.029317,0.027144,0.022166,0.275341,0.034148,0.020735,0.0
min,-1.572012e+00,-3.533581e+00,-1.426842e-01,-2.180971e-01,-3.156986e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,-7.777344e-01,-4.176987e-01,-1.426842e-01,-2.180971e-01,-3.161491e-02,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0
50%,-1.278708e-01,-2.821351e-02,-1.426842e-01,-2.180971e-01,-3.161491e-02,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0
75%,6.664070e-01,7.507569e-01,-1.426842e-01,-2.180971e-01,3.690737e-01,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0
max,3.699104e+00,2.308698e+00,1.304137e+01,9.133217e+00,4.696510e+00,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [35]:
test_nd.head(5)

,age,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad_and_Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,native-country_Holand-Netherlands
1,-0.994356,-1.196669,-0.142684,-0.218097,-0.031615,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,-0.055664,-0.417699,-0.142684,-0.218097,0.769762,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,-0.777734,0.750757,-0.142684,-0.218097,-0.031615,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0.377579,-0.028214,0.870916,-0.218097,-0.031615,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
5,-1.499805,-0.028214,-0.142684,-0.218097,-0.832992,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


### Dicision Tree

#### C5.0

##### Model Constructure

In [36]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

utils = importr('utils')
## 載入C50套件
C50 = importr('C50')
robjects.r('library(C50)')
partykit = importr("partykit")
robjects.r('Sys.setlocale("LC_ALL", "en_US.UTF-8")')

pandas2ri.activate()

## 將前處理的訓練與測試資料指派給r運算時的變數
r_train_data = pandas2ri.py2rpy(train_nd)
r_test_data = pandas2ri.py2rpy(test_nd)

robjects.r.assign("r_train_data", r_train_data)
robjects.r.assign("r_test_data", r_test_data)

# 在 R 中建立 C5.0 決策樹模型

robjects.r('col<-r_test_data$"income"')

array([0, 0, 1, ..., 0, 0, 1], dtype=int32)

In [37]:
# robjects.r('formula <- income ~ age + capital-gain + capital-loss + education-num + hours-per-week + marital-status_Divorced + marital-status_Married-AF-spouse + marital-status_Married-civ-spouse + marital-status_Married-spouse-absent + marital-status_Never-married + marital-status_Separated + marital-status_Widowed + native-country_Cambodia + native-country_Canada + native-country_China + native-country_Columbia + native-country_Cuba + native-country_Dominican-Republic + native-country_Ecuador + native-country_El-Salvador + native-country_England + native-country_France + native-country_Germany + native-country_Greece + native-country_Guatemala + native-country_Haiti + native-country_Holand-Netherlands + native-country_Honduras + native-country_Hong + native-country_Hungary + native-country_India + native-country_Iran + native-country_Ireland + native-country_Italy + native-country_Jamaica + native-country_Japan + native-country_Laos + native-country_Mexico + native-country_Nicaragua + native-country_Outlying-US(Guam-USVI-etc) + native-country_Peru + native-country_Philippines + native-country_Poland + native-country_Portugal + native-country_Puerto-Rico + native-country_Scotland + native-country_South + native-country_Taiwan + native-country_Thailand + native-country_Trinadad&Tobago + native-country_United-States + native-country_Vietnam + native-country_Yugoslavia + occupation_Adm-clerical + occupation_Armed-Forces + occupation_Craft-repair + occupation_Exec-managerial + occupation_Farming-fishing + occupation_Handlers-cleaners + occupation_Machine-op-inspct + occupation_Other-service + occupation_Priv-house-serv + occupation_Prof-specialty + occupation_Protective-serv + occupation_Sales + occupation_Tech-support + occupation_Transport-moving + race_Amer-Indian-Eskimo + race_Asian-Pac-Islander + race_Black + race_Other + race_White + relationship_Husband + relationship_Not-in-family + relationship_Other-relative + relationship_Own-child + relationship_Unmarried + relationship_Wife + sex_Female + sex_Male + workclass_Federal-gov + workclass_Local-gov + workclass_Never-worked + workclass_Private + workclass_Self-emp-inc + workclass_Self-emp-not-inc + workclass_State-gov + workclass_Without-pay')

robjects.r('colnames(r_train_data)[colnames(r_train_data) == "income"] <- "class"')
robjects.r('r_train_data$class <- as.factor(r_train_data$class)')
robjects.r('subset_data <- r_train_data[, !colnames(r_train_data) %in% "class"]')

robjects.r('model <- C5.0(x = subset_data, y = r_train_data$class, trials = 8, control = C5.0Control(CF = FALSE, label = "dick"))')

model = robjects.globalenv['model']
robjects.r('print(summary(model))')


Call:
C5.0.default(x = subset_data, y = r_train_data$class, trials = 8, control
 = C5.0Control(CF = FALSE, label = "dick"))


C5.0 [Release 2.07 GPL Edition]  	Thu Oct 26 17:35:16 2023
-------------------------------

Class specified by attribute `dick'

Read 32537 cases (89 attributes) from undefined.data

-----  Trial 0:  -----

Decision tree:

capital-gain > 0.781088: 1 (1399/20)
capital-gain <= 0.781088:
:...marital-status_Married-civ-spouse <= 0:
    :...capital-loss <= 5.255904: 0 (17155/789)
    :   capital-loss > 5.255904: 1 (101/41)
    marital-status_Married-civ-spouse > 0:
    :...capital-gain > 0.5389335: 1 (121/5)
        capital-gain <= 0.5389335:
        :...capital-loss > 4.154428:
            :...capital-loss <= 4.695242: 1 (585/14)
            :   capital-loss > 4.695242:
            :   :...capital-loss <= 5.149229: 0 (63)
            :       capital-loss > 5.149229:
            :       :...education-num <= 0.7459131: 0 (58/21)
            :           education-num 

output,[RTYPES.STRSXP]
call,[RTYPES.LANGSXP]


##### Performance test

In [46]:
# robjects.r('colnames(r_test_data)[colnames(r_test_data) == "income"] <- "class"')
robjects.r('subset_data_test <- r_test_data[, !colnames(r_test_data) %in% "income"]')
r_predictions = robjects.r('''
    confmatrix = function(Y,Ypred)
    {
        tab = table(Y,Ypredict=Ypred)
        print(tab)
        p = sum(diag(tab))/sum(tab)*100
        cat("\n\n預測正確率 = ",p,"% \n")
    }
    # 將模型套用在測試數據                 
    predictions <- predict(model, newdata = subset_data_test)         
    freq1 <-table(predictions, col)
                           
    accuracy<-sum(diag(freq1)/sum(freq1))
    print(accuracy)
    print(freq1)
   
    precision <- freq1[,1][1]/(freq1[,1][1]+freq1[,1][2])
    recall <- freq1[,1][1]/(freq1[,1][1]+freq1[,2][2])
    f1 <- (2*recall*precision)/(precision+recall)
                           
    cat("Precision:",precision,"\n")
    cat("Recall:",recall,"\n")
    cat("F1-score:",f1,"\n")

    #confmatrix(subset_data_test, predictions$income)
''')

[1] 0.8629884
           col
predictions     0     1
          0 11423  1223
          1  1007  2623
Precision: 0.9189863 
Recall: 0.8132564 
F1-score: 0.8628947 


##### 匯出Excel

In [ ]:
predictions = robjects.globalenv['predictions']

with (robjects.default_converter + pandas2ri.converter).context():
  pd_from_r_df = robjects.conversion.get_conversion().rpy2py(predictions)

from openpyxl import Workbook
#產出Excel(Test data)
wb = Workbook()
ws = wb.active
ws.append(['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income','Predict result'])

for i in range(len(pd_from_r_df)):
    if pd_from_r_df[i] == '0':
        result = '<=50K.'
    else:
        result = '>50K.'
    #將現在loop到原始資料的列轉為list
    li = test_nd.iloc[i,:].tolist()
    #
    li.append(result)
    ws.append(li)
wb.save('adult_c50_test.xlsx')
# predictions = np.array(predictions)

print(pd_from_r_df)